# Imports y cargar dataset

Fuente: https://www.kaggle.com/datasets/osmi/mental-health-in-tech-survey

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
!pip install kagglehub[pandas-datasets]

In [ ]:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "survey.csv"

# Load the latest version
df = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "osmi/mental-health-in-tech-survey",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

<ipython-input-3-028f5efd1efe>:9: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df = kagglehub.load_dataset(


First 5 records:              Timestamp  Age  Gender         Country state self_employed  \
0  2014-08-27 11:29:31   37  Female   United States    IL           NaN   
1  2014-08-27 11:29:37   44       M   United States    IN           NaN   
2  2014-08-27 11:29:44   32    Male          Canada   NaN           NaN   
3  2014-08-27 11:29:46   31    Male  United Kingdom   NaN           NaN   
4  2014-08-27 11:30:22   31    Male   United States    TX           NaN   

  family_history treatment work_interfere    no_employees  ...  \
0             No       Yes          Often            6-25  ...   
1             No        No         Rarely  More than 1000  ...   
2             No        No         Rarely            6-25  ...   
3            Yes       Yes          Often          26-100  ...   
4             No        No          Never         100-500  ...   

                leave mental_health_consequence phys_health_consequence  \
0       Somewhat easy                        No             

In [ ]:
df.shape

(1259, 27)

In [ ]:
df.columns

Index(['Timestamp', 'Age', 'Gender', 'Country', 'state', 'self_employed',
       'family_history', 'treatment', 'work_interfere', 'no_employees',
       'remote_work', 'tech_company', 'benefits', 'care_options',
       'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'comments'],
      dtype='object')

In [ ]:
df['mental_health_consequence'].unique()

array(['No', 'Maybe', 'Yes'], dtype=object)

In [ ]:
df = df.drop(columns=['state']) #He considerado no utilizarla ya que solo esta disponible para EEUU

In [ ]:
df['self_employed'] = df['self_employed'].fillna(df['self_employed'].mode()[0])
df['work_interfere'] = df['work_interfere'].fillna(df['work_interfere'].mode()[0])


In [ ]:
df['Age'].unique()

array([         37,          44,          32,          31,          33,
                35,          39,          42,          23,          29,
                36,          27,          46,          41,          34,
                30,          40,          38,          50,          24,
                18,          28,          26,          22,          19,
                25,          45,          21,         -29,          43,
                56,          60,          54,         329,          55,
       99999999999,          48,          20,          57,          58,
                47,          62,          51,          65,          49,
             -1726,           5,          53,          61,           8,
                11,          -1,          72])

In [ ]:
df['Gender'].unique()

array(['Female', 'M', 'Male', 'male', 'female', 'm', 'Male-ish', 'maile',
       'Trans-female', 'Cis Female', 'F', 'something kinda male?',
       'Cis Male', 'Woman', 'f', 'Mal', 'Male (CIS)', 'queer/she/they',
       'non-binary', 'Femake', 'woman', 'Make', 'Nah', 'All', 'Enby',
       'fluid', 'Genderqueer', 'Female ', 'Androgyne', 'Agender',
       'cis-female/femme', 'Guy (-ish) ^_^', 'male leaning androgynous',
       'Male ', 'Man', 'Trans woman', 'msle', 'Neuter', 'Female (trans)',
       'queer', 'Female (cis)', 'Mail', 'cis male', 'A little about you',
       'Malr', 'p', 'femail', 'Cis Man',
       'ostensibly male, unsure what that really means'], dtype=object)

In [ ]:
def clean_age(age, default_age):
    if pd.isna(age):
        return default_age
    elif age < 18 or age > 80:
        return default_age
    else:
        return age

# Completo con la mediana en las edades incorrectas para evitar perder muchos datos
median_age = int(df['Age'][(df['Age'] >= 18) & (df['Age'] <= 80)].median())

print(median_age)

df['Age'] = df['Age'].apply(lambda x: clean_age(x, default_age=median_age))


31


In [ ]:
male = [
    'Male', 'male', 'M', 'm', 'Man', 'Male ', 'Male-ish',
    'Cis Male', 'cis male', 'Cis Man',
    'Mal', 'Malr', 'msle', 'maile', 'Make', 'Mail', 'Male (CIS)'
]

female = [
    'Female', 'female', 'F', 'f', 'Woman', 'Female ',
    'Cis Female', 'cis-female/femme', 'Female (cis)',
    'Femake', 'femail','woman'
]

other = [
    'Trans-female', 'Trans woman', 'Female (trans)', 'Enby', 'fluid', 'non-binary',
    'Genderqueer', 'Genderqueer', 'Agender', 'Androgyne',
    'Guy (-ish) ^_^', 'male leaning androgynous', 'Neuter',
    'queer', 'queer/she/they', 'something kinda male?', 'ostensibly male, unsure what that really means',
    'All', 'A little about you', 'Nah', 'p'
]

df['Gender'].replace(to_replace = other, value = 'other', inplace=True)
df['Gender'].replace(to_replace = male, value = 'M', inplace=True)
df['Gender'].replace(to_replace = female, value = 'F', inplace=True)


<ipython-input-12-710ebf07e8c0>:21: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Gender'].replace(to_replace = other, value = 'other', inplace=True)


In [ ]:
df['Gender'].unique()

array(['F', 'M', 'other'], dtype=object)

In [ ]:
df.columns

Index(['Timestamp', 'Age', 'Gender', 'Country', 'self_employed',
       'family_history', 'treatment', 'work_interfere', 'no_employees',
       'remote_work', 'tech_company', 'benefits', 'care_options',
       'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'comments'],
      dtype='object')

In [ ]:
summary = pd.DataFrame({
    'columna': df.columns,
    'n_valores_unicos': [df[col].nunique() for col in df.columns],
    'valores_unicos': [df[col].unique() for col in df.columns]
})
summary

,columna,n_valores_unicos,valores_unicos
0,Timestamp,1246,"[2014-08-27 11:29:31, 2014-08-27 11:29:37, 201..."
1,Age,45,"[37, 44, 32, 31, 33, 35, 39, 42, 23, 29, 36, 2..."
2,Gender,3,"[F, M, other]"
3,Country,48,"[United States, Canada, United Kingdom, Bulgar..."
4,self_employed,2,"[No, Yes]"
5,family_history,2,"[No, Yes]"
6,treatment,2,"[Yes, No]"
7,work_interfere,4,"[Often, Rarely, Never, Sometimes]"
8,no_employees,6,"[6-25, More than 1000, 26-100, 100-500, 1-5, 5..."
9,remote_work,2,"[No, Yes]"


In [ ]:
df.columns

Index(['Timestamp', 'Age', 'Gender', 'Country', 'self_employed',
       'family_history', 'treatment', 'work_interfere', 'no_employees',
       'remote_work', 'tech_company', 'benefits', 'care_options',
       'wellness_program', 'seek_help', 'anonymity', 'leave',
       'mental_health_consequence', 'phys_health_consequence', 'coworkers',
       'supervisor', 'mental_health_interview', 'phys_health_interview',
       'mental_vs_physical', 'obs_consequence', 'comments'],
      dtype='object')

In [ ]:
df.to_csv("df_procesed.csv", index=False)

# 1. Gráfico inicial


In [ ]:
treatment_counts = df['treatment'].value_counts().reset_index()
treatment_counts.columns = ['treatment', 'count']
treatment_counts['percentage'] = round((treatment_counts['count'] / treatment_counts['count'].sum()) * 100, 2)
treatment_counts.to_csv("treatment.csv", index=False)
treatment_counts

,treatment,count,percentage
0,Yes,637,50.6
1,No,622,49.4


# 2. Perfil demográfico



## Genero

In [ ]:
df_temp = df[df['treatment'].isin(['Yes', 'No'])].copy()

grupos = {
    'M': 'Male',
    'F': 'Female',
    'other': 'Other / Non-binary'
}

resultados = []
for codigo, nombre in grupos.items():
    datos = df_temp[df_temp['Gender'] == codigo]['treatment']
    total = len(datos)
    if total > 0:
        yes_pct = (datos == 'Yes').mean() * 100
        no_pct = (datos == 'No').mean() * 100
        resultados.append({'group': nombre, 'category': 'Yes', 'percentage': round(yes_pct, 2)})
        resultados.append({'group': nombre, 'category': 'No', 'percentage': round(no_pct, 2)})

treatment_by_gender = pd.DataFrame(resultados)

treatment_by_gender.to_csv("treatment_by_gender.csv", index=False)
treatment_by_gender

,group,category,percentage
0,Male,Yes,45.41
1,Male,No,54.59
2,Female,Yes,68.83
3,Female,No,31.17
4,Other / Non-binary,Yes,80.95
5,Other / Non-binary,No,19.05


## Edad

In [ ]:
df_temp = df[df['treatment'].isin(['Yes', 'No']) & df['Age'].notna()].copy()

treatment_by_age = []

for age in sorted(df_temp['Age'].unique()):
    datos = df_temp[df_temp['Age'] == age]['treatment']
    total = len(datos)

    yes_pct = (datos == 'Yes').mean() * 100
    no_pct = (datos == 'No').mean() * 100

    treatment_by_age.append({'Age': age, 'Tratamiento': 'Yes', 'Percentage': round(yes_pct, 2)})
    treatment_by_age.append({'Age': age, 'Tratamiento': 'No', 'Percentage': round(no_pct, 2)})

treatment_by_age = pd.DataFrame(treatment_by_age)

treatment_by_age.to_csv("treatment_by_age.csv", index=False)
treatment_by_age

,Age,Tratamiento,Percentage
0,18,Yes,42.86
1,18,No,57.14
2,19,Yes,44.44
3,19,No,55.56
4,20,Yes,33.33
...,...,...,...
85,62,No,100.00
86,65,Yes,0.00
87,65,No,100.00
88,72,Yes,100.00


# 3. Empresas



## Tamaño de empresa

In [ ]:
df_temp = df[df['treatment'].isin(['Yes', 'No']) & df['no_employees'].notna()].copy()

treatment_by_company_size = []

for size in sorted(df_temp['no_employees'].unique()):
    datos = df_temp[df_temp['no_employees'] == size]['treatment']
    total = len(datos)

    yes_pct = (datos == 'Yes').mean() * 100
    no_pct = (datos == 'No').mean() * 100

    treatment_by_company_size.append({
        'no_employees': size,
        'Yes': round(yes_pct, 2),
        'No': round(no_pct, 2)
    })

treatment_by_company_size = pd.DataFrame(treatment_by_company_size)

treatment_by_company_size.to_csv("treatment_by_company_size.csv", index=False)
treatment_by_company_size

,no_employees,Yes,No
0,1-5,56.17,43.83
1,100-500,53.98,46.02
2,26-100,51.90,48.10
3,500-1000,45.00,55.00
4,6-25,44.14,55.86
5,More than 1000,51.77,48.23


# Mundo tech


In [ ]:
df_temp = df[
    df['treatment'].isin(['Yes', 'No']) &
    df['tech_company'].isin(['Yes', 'No'])
].copy()

treatment_by_tech_company = []

for company_type in ['Yes', 'No']:
    grupo = df_temp[df_temp['tech_company'] == company_type]['treatment']
    total = len(grupo)

    yes_pct = (grupo == 'Yes').mean() * 100
    no_pct = (grupo == 'No').mean() * 100

    treatment_by_tech_company.append({
        'tech_company': company_type,
        'treatment': 'Yes',
        'count': (grupo == 'Yes').sum(),
        'percentage': round(yes_pct, 2)
    })
    treatment_by_tech_company.append({
        'tech_company': company_type,
        'treatment': 'No',
        'count': (grupo == 'No').sum(),
        'percentage': round(no_pct, 2)
    })

treatment_by_tech_company = pd.DataFrame(treatment_by_tech_company)

treatment_by_tech_company.to_csv("treatment_vs_tech_company.csv", index=False)
treatment_by_tech_company

,tech_company,treatment,count,percentage
0,Yes,Yes,514,49.85
1,Yes,No,517,50.15
2,No,Yes,123,53.95
3,No,No,105,46.05


## Consecuencias

In [ ]:
df_temp = df[(df['treatment'].isin(['Yes', 'No'])) & (df['tech_company'].isin(['Yes', 'No']))].copy()

variables = [
    'mental_health_consequence',
    'mental_health_interview',
    'obs_consequence',
    'coworkers',
    'supervisor'
]

labels = [
    'Padeció consecuencias',
    'Comodidad en entrevistas',
    'Observó consecuencias',
    'Comodidad con compañeros',
    'Comodidad con supervisores'
]

tech = []
notech = []

for var in variables:
    tech_data = df_temp[df_temp['tech_company'] == 'Yes'][var]
    notech_data = df_temp[df_temp['tech_company'] == 'No'][var]

    tech_yes = (tech_data == 'Yes').mean() * 100
    notech_yes = (notech_data == 'Yes').mean() * 100

    tech.append(round(tech_yes, 2))
    notech.append(round(notech_yes, 2))

mental_health_consequences_by_sector = pd.DataFrame({
    'Variable': labels,
    'Tech': tech,
    'No-Tech': notech
})

mental_health_consequences_by_sector.to_csv("consecuencias_salud_mental_tech_vs_notech.csv", index=False)
mental_health_consequences_by_sector

,Variable,Tech,No-Tech
0,Padeció consecuencias,21.82,29.39
1,Comodidad en entrevistas,3.88,1.75
2,Observó consecuencias,13.58,19.30
3,Comodidad con compañeros,19.11,12.28
4,Comodidad con supervisores,42.10,35.96


## Beneficios

In [ ]:
df_temp = df[df['tech_company'].isin(['Yes', 'No'])].copy()

variables = ['wellness_program', 'seek_help', 'anonymity', 'remote_work']
labels = ['Programa de bienestar', 'Recursos para buscar ayuda', 'Anonimato al pedir ayuda', 'Teletrabajo']

tech = []
notech = []

for var in variables:
    tech_data = df_temp[df_temp['tech_company'] == 'Yes'][var]
    notech_data = df_temp[df_temp['tech_company'] == 'No'][var]

    tech_pct = (tech_data == 'Yes').mean() * 100
    notech_pct = (notech_data == 'Yes').mean() * 100

    tech.append(round(tech_pct, 2))
    notech.append(round(notech_pct, 2))

mental_health_resources= pd.DataFrame({
    'Variable': labels,
    'Tech': tech,
    'No-Tech': notech
})

mental_health_resources.to_csv("comparativa_tech_vs_notech.csv", index=False)
mental_health_resources

,Variable,Tech,No-Tech
0,Programa de bienestar,15.71,29.39
1,Recursos para buscar ayuda,17.56,30.26
2,Anonimato al pedir ayuda,28.90,33.77
3,Teletrabajo,32.69,17.11


# Suicidio

Fuente: https://www.kaggle.com/datasets/armanzhalgasbayev/world-countries-rankings-by-suicide-rate-2023

In [ ]:
# Install dependencies as needed:
# pip install kagglehub[pandas-datasets]
import kagglehub
from kagglehub import KaggleDatasetAdapter

# Set the path to the file you'd like to load
file_path = "world_suicide_rate_2023.csv"

# Load the latest version
df_suicide = kagglehub.load_dataset(
  KaggleDatasetAdapter.PANDAS,
  "armanzhalgasbayev/world-countries-rankings-by-suicide-rate-2023",
  file_path,
  # Provide any additional arguments like
  # sql_query or pandas_kwargs. See the
  # documenation for more information:
  # https://github.com/Kaggle/kagglehub/blob/main/README.md#kaggledatasetadapterpandas
)

print("First 5 records:", df.head())

<ipython-input-24-5754e9971bd3>:10: DeprecationWarning: load_dataset is deprecated and will be removed in a future version.
  df_suicide = kagglehub.load_dataset(


100%|██████████| 6.71k/6.71k [00:00<00:00, 6.70MB/s]

First 5 records:              Timestamp  Age Gender         Country self_employed  \
0  2014-08-27 11:29:31   37      F   United States            No   
1  2014-08-27 11:29:37   44      M   United States            No   
2  2014-08-27 11:29:44   32      M          Canada            No   
3  2014-08-27 11:29:46   31      M  United Kingdom            No   
4  2014-08-27 11:30:22   31      M   United States            No   

  family_history treatment work_interfere    no_employees remote_work  ...  \
0             No       Yes          Often            6-25          No  ...   
1             No        No         Rarely  More than 1000          No  ...   
2             No        No         Rarely            6-25          No  ...   
3            Yes       Yes          Often          26-100          No  ...   
4             No        No          Never         100-500         Yes  ...   

                leave mental_health_consequence phys_health_consequence  \
0       Somewhat easy         

In [ ]:
df_suicide.columns

Index(['Country', 'All', 'Female', 'Male', 'M/F', '2000', 'Change%'], dtype='object')

In [ ]:
# Paises que aparecen en ambos datasets
selected_countries = sorted(set(df['Country']) & set(df_suicide['Country']))
selected_countries

['Australia',
 'Austria',
 'Belgium',
 'Bosnia and Herzegovina',
 'Brazil',
 'Bulgaria',
 'Canada',
 'China',
 'Colombia',
 'Costa Rica',
 'Croatia',
 'Czech Republic',
 'Denmark',
 'Finland',
 'France',
 'Georgia',
 'Germany',
 'Greece',
 'Hungary',
 'India',
 'Ireland',
 'Israel',
 'Italy',
 'Japan',
 'Latvia',
 'Mexico',
 'Moldova',
 'Netherlands',
 'New Zealand',
 'Nigeria',
 'Norway',
 'Philippines',
 'Poland',
 'Portugal',
 'Romania',
 'Russia',
 'Singapore',
 'Slovenia',
 'South Africa',
 'Spain',
 'Sweden',
 'Switzerland',
 'Thailand',
 'United Kingdom',
 'United States',
 'Uruguay',
 'Zimbabwe']

In [ ]:
df_temp = df[
    (df['tech_company'] == 'Yes') &
    (df['Country'].isin(selected_countries)) &
    (df['treatment'] == 'Yes')
].copy()

agrupado = []

paises = df_temp['Country'].unique()
for country in sorted(paises):
    # Total de personas tech en encuesta de ese país
    total = len(df[(df['tech_company'] == 'Yes') & (df['Country'] == country)])
    tratados = len(df_temp[df_temp['Country'] == country])

    if total > 4:
        score = (tratados / total) * np.sqrt(total)
        agrupado.append({
            'Country': country,
            'total': total,
            'treated': tratados,
            'score': score
        })

agg = pd.DataFrame(agrupado)

merged = pd.merge(agg, df_suicide[['Country', 'All', 'Change%']], on='Country', how='inner')

# Escalado del score de tratamiento
score_min = merged['score'].min()
score_max = merged['score'].max()
merged['score_scaled'] = ((merged['score'] - score_min) / (score_max - score_min) * 100).round(2)

# Escalado de la tasa de suicidios
suicide_min = merged['All'].min()
suicide_max = merged['All'].max()
merged['suicide_scaled'] = ((merged['All'] - suicide_min) / (suicide_max - suicide_min) * 100).round(2)

merged.to_csv("suicide_vs_treat_country.csv", index=False)
merged

,Country,total,treated,score,All,Change%,score_scaled,suicide_scaled
0,Australia,17,11,2.667892,11.3,-0.05,16.84,68.63
1,Brazil,6,2,0.816497,6.4,0.43,2.80,20.59
2,Canada,56,24,3.207135,10.3,-0.03,20.93,58.82
3,France,12,2,0.577350,9.7,-0.39,0.99,52.94
4,Germany,40,17,2.687936,8.3,-0.26,16.99,39.22
5,India,9,3,1.000000,12.9,-0.32,4.19,84.31
6,Ireland,26,12,2.353394,8.9,-0.26,14.46,45.10
7,Italy,5,1,0.447214,4.3,-0.22,0.00,0.00
8,Netherlands,25,9,1.800000,9.3,0.15,10.26,49.02
9,New Zealand,7,4,1.511858,10.3,-0.17,8.07,58.82


# Wordcloud

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import nltk

nltk.download('stopwords')

# Lista de stopwords
stop_words = stopwords.words('english')

comments = df['comments'].dropna()
vectorizer = CountVectorizer(stop_words=stop_words)
X = vectorizer.fit_transform(comments)

# Frecuencias
word_freq = X.toarray().sum(axis=0)
vocab = vectorizer.get_feature_names_out()

keywords_df = pd.DataFrame({'keyword': vocab, 'frequency': word_freq})
keywords_df = keywords_df.sort_values(by='frequency', ascending=False).reset_index(drop=True)
keywords_df.to_csv("word_cloud.csv", index=False)
keywords_df.head()


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


,keyword,frequency
0,health,114
1,mental,103
2,work,50
3,employer,45
4,issues,40
